# Create Coincident Load All Main Meters
Finds usage for all main meters during the GRE monthly coincidental load.  This should align closely with the GRE numbers.

In [0]:
%run ../../Utilities/ConfigUtilities

In [0]:
# Set up the environment using a function in ConfigUtilties.
set_spark_config()

In [0]:
from pyspark.sql.functions import col, concat, lit, sum

debug = 1

In [0]:
# Read the coincidental load information with index information.
coincid_load_df = spark.read.format("delta").load(COINCIDENTAL_LOAD_INDEX_PATH)

if debug:
  display(coincid_load_df)

In [0]:
# Read the hourly meter data for the main meters (no submeters)
hourly_main_df = spark.read.format("delta").load(MDM_HOURLY_NO_SUBMETERS_PATH)

if debug:
    display(hourly_main_df)

In [0]:
# Filter the hourly data to the coincidental peak hour by joining with the coincidental peak dataframe.
hourly_main_coincid_df = hourly_main_df.join(coincid_load_df, hourly_main_df.EndMeterSampleIndex == coincid_load_df.MeterSampleIndex)

hourly_main_coincid_df = hourly_main_coincid_df.drop('MeterSampleIndex', 'LocalYear', 'LocalMonth', 'LocalDay', 'HourEnding')

if debug:
    display(hourly_main_coincid_df)

In [0]:
# Get totals for the coincidental peak slot for all meters.
coincid_total_df = hourly_main_coincid_df.groupBy("CoincidYear", "CoincidMonth").agg(sum("HourlyAMIValue").alias("TotalCPHourlyAMIValue"), sum("HourlyVEEValue").alias("TotalCPHourlyVEEValue"))

coincid_total_df = coincid_total_df.withColumn("YearMonth", concat(coincid_total_df.CoincidYear, lit("-"), coincid_total_df.CoincidMonth))

if debug:
    display(coincid_total_df)


In [0]:
# Save this total for comparison with the GRE total and the load control total.
coincid_total_df.write.format("delta").mode("overwrite").save(COINCIDENTAL_PEAK_ALL_METER_TOTAL_PATH) 


In [0]:
# Clean up the delta history.
spark.sql(f"VACUUM '{COINCIDENTAL_PEAK_ALL_METER_TOTAL_PATH}'")